In [8]:
%cd

/root


##  Setup & Clone Repo

In [10]:
#!git clone https://github.com/Sruthi-Reddy-B/Object-detection-with-pose-visualization.git
%cd Object-detection-with-pose-visualization
!pip install -r requirements.txt

import os
os.makedirs('results/detections', exist_ok=True)
os.makedirs('results/visualization', exist_ok=True)


/root/Object-detection-with-pose-visualization


## Download Demo Image & Create Dummy Pose

In [11]:
!mkdir -p data/images data/poses
!wget -q https://github.com/ultralytics/assets/releases/download/v0.0.0/bus.jpg -O data/images/object_1.jpg

import pandas as pd
pose_data = pd.DataFrame({
    'image': ['object_1.jpg'],
    'x': [120.5], 'y': [85.3], 'z': [0.55],
    'roll': [10.0], 'pitch': [5.0], 'yaw': [45.0]
})
pose_data.to_csv("data/poses/pose_labels.csv", index=False)


## Train YOLO Model (Optional Demo)

In [ ]:
# This uses src/train.py
#!python src/train.py


## Run Inference & Visualize 6D Pose


In [12]:
# Download YOLOv8 pre-trained weights if not present
import os
if not os.path.exists("yolov8n.pt"):
    !wget -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt -O yolov8n.pt



In [13]:
!python src/inference.py


Traceback (most recent call last):
  File "/root/Object-detection-with-pose-visualization/src/inference.py", line 23, in <module>
    results = model.predict(source=IMAGE_PATH, save=False)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 557, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/predictor.py", line 230, in __call__
    return list(self.stream_inference(source, model, *args, **kwargs))  # merge list of Result into one
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/_contextlib.py", line 38, in generator_context
    response = gen.send(None)
          

## Display Results in Notebook

In [ ]:
import cv2
import matplotlib.pyplot as plt

img_pose = cv2.imread('results/visualization/pose_axes.jpg')
plt.figure(figsize=(6,6))
plt.imshow(cv2.cvtColor(img_pose, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Estimated 6D Pose Visualization")
plt.show()
